# IIOT 工業4.0 - 以 Python 實作工廠監控

## 自我介紹

- Malo, 高雄Python社群共同創辨人
- 目前工作為太陽能監控系統開發
- 韌體工程師 --> 全端工程師

# Agenda

- 工廠監控

    - 今天的主軸：如何在Raspberry Pi上使用python和PLC、電表溝通, 並把資料送上雲端

- Modbus
    - 何謂Modbus?
    - modbus格式介紹
    - modbus常被採用的原因
        
- Modbus套件
    - 可用的套件
    - 安裝
    - 操作

- PLC
    - 何謂PLC
    - 如何通訊、控制
    - 來看看PLC的Modbus點表
    - Live Demo




- Power Meter
    - 何謂power meter
    - 來看看Power Meter的點表
    - Live Demo
    
- MQTT - 即時性應用
    - Mosquitto
    - paho
    
- The End!?
 
- Q&A

----

## 工廠監控

----


### 今天的主軸：如何使用python和PLC、電表、逆變器溝通, 並把資料送上雲端

- 範例可以在windows、Linux、Raspberry pi、BeagleBone Black上執行
    
- ![PLC_Meter](image/plc_meter.png)

## Live Demo

先來看看我們使用Python可以達到的效果

- Python + PLC + Power Meter + MQTT + Linear MQTT Dashboard

![mqtt](image/mqtt.png)

----

## 工廠監控的現況

先看看業界現在怎麼做…

----


### 很常見的方式

- SCADA + HMI + PLC + IO module

- PLC + SCADA


### 系統小一點的話

- HMI + PLC + IO module




### 喜歡自已來!?

- 當然是自己開發囉! VB.Net, C#, C 各種語言不拘!


### 為何SCADA這麼常出現?

- 因為工業控制需要快速又穩定的RAD工具，讓每個人都能做出一定水準之上的系統

![Scada_std_anim_no_lang](image/Scada_std_anim_no_lang.gif)


----

## Modbus

一個工業控制一定不能錯過的協定

----


### 何謂 Modbus?
一種工業控制中很常用的通訊協定

- 為何介紹Modbus

![協定文字雲](image/protocol_cloud.png)
       

### modbus格式介紹 

[參考](http://gridconnect.com/blog/tag/modbus-rtu/)

- Modbus/RTU: [start time] [Address 8bits + Function 8bits + Data Nx8bits + CRC 16bits] [End time]

- Modbus/TCP: [header 6byte + Address 8bits + Function 8bits + Data Nx8bits]

- Modbus/ASCII: 現在比較少人用，跳過不講

![格式](image/MODBUS-Frame.png)
        


### modbus常被採用的原因
        
- 公開發表並且無版稅要求

- 相對容易的工業網路部署

- 協定格式簡單，極省資源


----

## Modbus套件

----



### 網路上較多人提到的三個套件

[performance比較](https://stackoverflow.com/questions/17081442/python-modbus-library)

- modbus-tk: Modbus/RTU, Modbus/TCP

- pymodbus: 據說實作最完整，但使用資源相對的多，相依套件也多

- MinimalModbus: Modbus/RTU, Modbus/ASCII



### Modbus-tk

個人推薦使用

- 安裝方式
    - pip install serial
    - pip install modbus_tk

- 操作
    - [Python與PLC共舞](https://github.com/maloyang/PLC-Python)
     

----

## PLC

工業控制常用的元素，[看看wiki怎麼說](https://zh.wikipedia.org/wiki/%E5%8F%AF%E7%BC%96%E7%A8%8B%E9%80%BB%E8%BE%91%E6%8E%A7%E5%88%B6%E5%99%A8)

----



### 通訊方式

- 自有協定

- <h3>Modbus</h3>

- CAN

- ...etc


### PLC的Modbus點表

![PLC點表](image/fatek_modbus_addr.png)


### live demo code : PLC
- [Live Demo](Modbus.ipynb)

In [1]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time


In [2]:
mbComPort = "COM8"   # your RS-485 port. for linux --> "/dev/ttyUSB0"
baudrate = 9600
databit = 8 #7, 8
parity = 'N' #N, O, E
stopbit = 1 #1, 2
mbTimeout = 100 # ms


In [3]:

mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 1
addr = 2 #base0 --> my 110V Led燈泡

for i in range(5):
    try:
        value = i%2
        #-- FC5: write multi-coils
        rr = master.execute(mbId, cst.WRITE_SINGLE_COIL, addr, output_value=value)
        print("Write(addr, value)=",  rr)

    except Exception as e:
        print("modbus test Error: " + str(e))

    time.sleep(2)

master._do_close()


Write(addr, value)= (2, 0)
Write(addr, value)= (2, 65280)
Write(addr, value)= (2, 0)
Write(addr, value)= (2, 65280)
Write(addr, value)= (2, 0)


True

----

## Power Meter

電力監控常見的元素

----



### Power Meter的點表

![Power Meter的點表](image/power_meter.gif)



### Power Meter的浮點數表示方式

![float](image/mb_float.png)


### live demo code: power meter

- [Live Demo](Modbus.ipynb)

In [4]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time
from struct import *

In [5]:
mbComPort = 'COM8' #your RS-485 port #'/dev/ttyUSB0' for linux(RPi3)
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

In [6]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 4
addr = 0x1000 # power-meter is base 0
# notice: meter not support FC6, only FC16

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 2)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        v_a_hi = rr[1]
        v_a_lo = rr[0]
        v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))
        print('v_a=', v_a)
        #struct.unpack(">f",'\x42\xd8\x6b\x8d')
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()


read value: (39507, 17124)
v_a= (114.3014144897461,)


True

### 讀回電錶的功率

In [7]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 4
addr = 0x1034 #kWh

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 2)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        hi = rr[1]
        lo = rr[0]
        kwh = unpack('>f', pack('>HH', hi, lo))
        print('kWh=', kwh)
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()


read value: (3077, 18333)
kWh= (80408.0390625,)


True

----

## MQTT - 即時性應用

已經可以採集資料了，來談談怎麼上傳雲端吧

----


### Mosquitto

一個broker套件，當然也可以做為client使用

- 以NB X260來說，4000多個連結沒有問題

![Mosquitto](image/Eclipse-Mosquitto-logo.png)


### paho

便利的MQTT client端套件

- [link](https://pypi.org/project/paho-mqtt/)
- install: `pip install paho-mqtt`

![paho](image/mqtt-paho-featured-image.jpg)


### live demo

- 雲端控制PLC為例


In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import paho.mqtt.client as mqtt  #import the client1
import time

import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time
from struct import *
import random

mbComPort = 'COM8' # for linux: '/dev/ttyUSB0'
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

def on_connect(client, userdata, flags, rc):
    m="Connected flags"+str(flags)+", result code "+str(rc)+", client_id  "+str(client)
    print(m)

    # first value OFF
    print('set light off!')
    control_light(0)
    client1.publish(topic,0)    

def on_message(client1, userdata, message):
    print("message received  "  ,str(message.payload.decode("utf-8")), message.topic, message.qos, message.retain)
    if message.topic == topic:
        my_message = str(message.payload.decode("utf-8"))
        print("set light: ", my_message)
        if my_message=='1' or my_message==1:
            control_light(1)
        else:
            control_light(0)

def on_log(client, userdata, level, buf):
    print("log: ",buf)

def control_light(value):
    mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
    master = modbus_rtu.RtuMaster(mb_port)
    master.set_timeout(mbTimeout/1000.0)

    mbId = 1
    addr = 2 #base0

    try:
        #-- FC5: write multi-coils
        rr = master.execute(mbId, cst.WRITE_SINGLE_COIL, addr, output_value=value)
        print("Write(addr, value)=%s" %(str(rr)))

    except Exception as e:#Exception, e:
        print("modbus test Error: " + str(e))

    master._do_close()


def read_power_meter():
    mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
    master = modbus_rtu.RtuMaster(mb_port)
    master.set_timeout(mbTimeout/1000.0)

    mbId = 4
    #[0x1000-0x1001]=VIn_a
    addr = 0x1000#4096

    v_a = None
    try:
        # FC3
        rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 4)
        print('read value:', rr)

        # convert to float:
        # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
        try:
            v_a_hi = rr[1]
            v_a_lo = rr[0]
            v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))
            print('v_a=', v_a)
        except Exception as e:#Exception, e:
            print(e)
    except Exception as e:#Exception, e:
        print("modbus test Error: " + str(e))

    master._do_close()
    if v_a==None:
        v_a=None
    else:
        v_a=v_a[0]
    return v_a

# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address= 'broker.hivemq.com' # "iot.eclipse.org"
topic = "malo-iot/light"
client1 = mqtt.Client()    #create new instance
client1.on_connect = on_connect        #attach function to callback
client1.on_message = on_message        #attach function to callback
#client1.on_log=on_log

time.sleep(1)
client1.connect(broker_address, 1883, 60)      #connect to broker
client1.subscribe(topic)

#client1.loop_forever()
# 有自己的while loop，所以call loop_start()，不用loop_forever
client1.loop_start()    #start the loop
time.sleep(2)
print("loop start")

while True:
    #v = read_power_meter()
    #print('V=%s, type(V)=%s' %(v, type(v)))
    
    v = random.randint(110, 115)
    client1.publish("malo-iot/voltage", v)
    time.sleep(2)


----

## The End!!

----


----

## Q&A

----

### 補充資料

## 太陽能電廠監控
----
![inv2cloud](image/inv2cloud.png)

In [ ]:
def read_inv(mb_id=1, port='/dev/ttyUSB0', br=9600, databit=8, parity='N', stopbit=1, timeout=1000):
    data = {'time':time.strftime("%Y-%m-%d %H:%M:%S"), 'total_energy':0, 't':[0], 'ac_vawf':[0,0,0,0],
        'dc_vaw':[0,0,0], 'error':[0]}

    try:
        mb_port = serial.Serial(port=port, baudrate=br, bytesize=databit, parity=parity, stopbits=stopbit)
        master = modbus_rtu.RtuMaster(mb_port)
        master.set_timeout(timeout/1000.0)

        #-- start to poll
        addr = 132-1
        # 132, 0: 2B, Daily Energy, Wh (IEEE32 float)
        # 134, 2: 2B, Total Energy, kWh
        # 144, 12: 2B, AC Voltage, V
        # 146, 14: 2B, AC Current, A
        # 148, 16: 2B, AC Power, W
        # 150, 18: 2B, AC Frequency, Hz
        # 152, 20: 2B, DC Power 1, W
        # 154, 22: 2B, DC Voltage 1, v
        # 156, 24: 2B, DC Current 1, A
        # 158, 26: 2B, DC Power 2, W
        # 160, 28: 2B, DC Voltage 2, V
        # 162, 30: 2B, DC Current 2, A
        # 164, 32: 2B, Temperature, oC
        for j in range(3): 
            try:
                rr = master.execute(mb_id, cst.ANALOG_INPUTS, addr, 34)

                today_energy = unpack('>f', pack('>HH', rr[1], rr[0]))[0]/1000

                ac_v = unpack('>f', pack('>HH', rr[13], rr[12]))[0]
                ac_a = unpack('>f', pack('>HH', rr[15], rr[14]))[0]
                ac_w = unpack('>f', pack('>HH', rr[17], rr[16]))[0]
                ac_f = unpack('>f', pack('>HH', rr[19], rr[18]))[0]

                dc_w = unpack('>f', pack('>HH', rr[21], rr[20]))[0]
                dc_v = unpack('>f', pack('>HH', rr[23], rr[22]))[0]
                dc_a = unpack('>f', pack('>HH', rr[25], rr[24]))[0]
                dc_w2 = unpack('>f', pack('>HH', rr[27], rr[26]))[0]
                dc_v2 = unpack('>f', pack('>HH', rr[29], rr[28]))[0]
                dc_a2 = unpack('>f', pack('>HH', rr[31], rr[30]))[0]
                
                data['today_energy'] = today_energy # kWh
                #- ac_vawf (for R, S, T)
                data['ac_vawf'] = [ac_v, ac_a, ac_w, ac_f] # V, A, W, Hz
                #- dc_vaw, (for MPPT1, MPPT2)
                data['dc_vaw'] = [dc_v, dc_a, dc_w, dc_v2, dc_a2, dc_w2] # V, A, W
                
                break #success-->exit to next
            except Exception as e:
                print('poll all error', e)

        master._do_close()
    except Exception as e:
        print("Error: " + str(e))

    return data

# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address= 'broker.hivemq.com' # "iot.eclipse.org"
topic = "malo-iot/light"
client1 = mqtt.Client()    #create new instance
#client1.on_connect = on_connect        #attach function to callback
#client1.on_message = on_message        #attach function to callback
#client1.on_log=on_log

time.sleep(1)
client1.connect(broker_address, 1883, 60)      #connect to broker
client1.subscribe(topic)

#client1.loop_forever()
# 有自己的while loop，所以call loop_start()，不用loop_forever
client1.loop_start()    #start the loop
time.sleep(2)
print("loop start")

if True:#while True:
    #v = read_power_meter()
    #print('V=%s, type(V)=%s' %(v, type(v)))
    
    #v = random.randint(110, 115)
    #client1.publish("malo-iot/voltage", v)

    inv_data = read_inv(22, 'COM6')
    print('inv_data: ', inv_data)
    client1.publish("malo-iot/today_energy", round(inv_data['today_energy'], 2))
    client1.publish("malo-iot/ac_v", round(inv_data['ac_vawf'][0], 2))
    client1.publish("malo-iot/ac_a", round(inv_data['ac_vawf'][1], 2))
    client1.publish("malo-iot/ac_w", round(inv_data['ac_vawf'][2], 2))
    client1.publish("malo-iot/ac_f", round(inv_data['ac_vawf'][3], 2))

    time.sleep(2)
